In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Projects/chinese_mom_simulator

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Projects/chinese_mom_simulator


In [2]:
!pip install -q transformers datasets trl peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.8/544.8 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 35.8 MB/s eta 0:00:00


In [3]:
# Loading and preparing dataset

import pandas as pd
from datasets import Dataset

# Load CSV
df = pd.read_csv("dataset/preprocessed_simulated_comments.csv")

# Combine columns into instruction-style format
def format_sample(row):
    return {
        "text": f"<|user|>\nPOST: {row['post']}\n<|assistant|>\n{row['comment']}"
    }

dataset = Dataset.from_pandas(df)
dataset = dataset.map(format_sample)
dataset = dataset.remove_columns([col for col in dataset.column_names if col != "text"])

# Split
dataset = dataset.train_test_split(test_size=0.05)


Map:   0%|          | 0/330 [00:00<?, ? examples/s]

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
model_name = "tiiuae/falcon-7b"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token  # Falcon has no pad token

from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,  # or torch.float16 if bfloat16 not supported
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [5]:
from peft import LoraConfig, get_peft_model, TaskType

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query_key_value"],  # Works well with Falcon
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()


trainable params: 2,359,296 || all params: 6,924,080,000 || trainable%: 0.0341


In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=50,
    learning_rate=1e-4,
    logging_steps=10,
    output_dir="./falcon-lora-chinese-mom",
    save_strategy="epoch",
    fp16=True,
    report_to="none"
)

In [7]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    peft_config=peft_config,
    args=training_args,
)
model.train()
trainer.train()


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Adding EOS to train dataset:   0%|          | 0/313 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/313 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2104 > 2048). Running this sequence through the model will result in indexing errors


Truncating train dataset:   0%|          | 0/313 [00:00<?, ? examples/s]

...

Tokenizing eval dataset:   0%|          | 0/17 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/17 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.


Step,Training Loss
10,2.504100
20,2.481800
30,2.407500
40,2.355600
50,2.330500
60,2.245800
70,2.308800
80,2.270200
90,2.272500
100,2.226200


TrainOutput(global_step=2000, training_loss=1.8120996565818788, metrics={'train_runtime': 5269.4662, 'train_samples_per_second': 2.97, 'train_steps_per_second': 0.38, 'total_flos': 3.905096728766922e+17, 'train_loss': 1.8120996565818788, 'epoch': 50.0})

In [8]:
model.save_pretrained("model/falcon7b-chinese-mom-lora")
tokenizer.save_pretrained("model/falcon7b-chinese-mom-lora")

('model/falcon7b-chinese-mom-lora/tokenizer_config.json',
 'model/falcon7b-chinese-mom-lora/special_tokens_map.json',
 'model/falcon7b-chinese-mom-lora/tokenizer.json')